## Using Recurrent neural networks (RNNs) to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

Using TensorFlow backend.


In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

C:\Users\Jun Kit\.conda\envs\ML\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 51s 2ms/step - loss: 0.6557 - accuracy: 0.6212 - val_loss: 0.5995 - val_accuracy: 0.6828
Epoch 2/10
25000/25000 [==============================] - 51s 2ms/step - loss: 0.5533 - accuracy: 0.7183 - val_loss: 0.5405 - val_accuracy: 0.7234
Epoch 3/10
25000/25000 [==============================] - 37s 1ms/step - loss: 0.4948 - accuracy: 0.7614 - val_loss: 0.5154 - val_accuracy: 0.7383
Epoch 4/10
25000/25000 [==============================] - 31s 1ms/step - loss: 0.4555 - accuracy: 0.7874 - val_loss: 0.4881 - val_accuracy: 0.7571
Epoch 5/10
25000/25000 [==============================] - 41s 2ms/step - loss: 0.4267 - accuracy: 0.8030 - val_loss: 0.4740 - val_accuracy: 0.7683
Epoch 6/10
25000/25000 [==============================] - 45s 2ms/step - loss: 0.4062 - accuracy: 0.8158 - val_loss: 0.4612 - val_accuracy: 0.7764
Epoch 7/10
25000/25000 [==============================] - 49s 2ms/st

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 6s 238us/step
Test score: 0.45011723630905154
Test accuracy: 0.787559986114502


## Exercise
### Your Turn

Now do it yourself:
- Prepare the data to use sequences of length 80 rather than length 30.  Did it improve the performance?
- Try different values of the "max_features".  Can you improve the performance?
- Try smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

C:\Users\Jun Kit\.conda\envs\ML\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 74s 3ms/step - loss: 0.6149 - accuracy: 0.6653 - val_loss: 0.5257 - val_accuracy: 0.7503
Epoch 2/10
25000/25000 [==============================] - 72s 3ms/step - loss: 0.4658 - accuracy: 0.7845 - val_loss: 0.4635 - val_accuracy: 0.7730
Epoch 3/10
25000/25000 [==============================] - 70s 3ms/step - loss: 0.3986 - accuracy: 0.8223 - val_loss: 0.4235 - val_accuracy: 0.8023
Epoch 4/10
25000/25000 [==============================] - 62s 2ms/step - loss: 0.3562 - accuracy: 0.8474 - val_loss: 0.4071 - val_accuracy: 0.8191
Epoch 5/10
25000/25000 [==============================] - 73s 3ms/step - loss: 0.3278 - accuracy: 0.8616 - val_loss: 0.3943 - val_accuracy: 0.8194
Epoch 6/10
25000/25000 [==============================] - 73s 3ms/step - loss: 0.3071 - accuracy: 0.8712 - val_loss: 0.4050 - val_accuracy: 0.8152
Epoch 7/10
25000/25000 [==============================] - 71s 3ms/st

C:\Users\Jun Kit\.conda\envs\ML\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 53s 2ms/step - loss: 0.6851 - accuracy: 0.5246 - val_loss: 0.6625 - val_accuracy: 0.5716
Epoch 2/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.6250 - accuracy: 0.6895 - val_loss: 0.6153 - val_accuracy: 0.6920
Epoch 3/10
25000/25000 [==============================] - 45s 2ms/step - loss: 0.5673 - accuracy: 0.7580 - val_loss: 0.5243 - val_accuracy: 0.7346
Epoch 4/10
25000/25000 [==============================] - 66s 3ms/step - loss: 0.4531 - accuracy: 0.7890 - val_loss: 0.4627 - val_accuracy: 0.7806
Epoch 5/10
25000/25000 [==============================] - 56s 2ms/step - loss: 0.4134 - accuracy: 0.8132 - val_loss: 0.4412 - val_accuracy: 0.7927
Epoch 6/10
25000/25000 [==============================] - 55s 2ms/step - loss: 0.3896 - accuracy: 0.8273 - val_loss: 0.4450 - val_accuracy: 0.7952
Epoch 7/10
25000/25000 [==============================] - 55s 2ms/st

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 51s 2ms/step - loss: 0.3303 - accuracy: 0.8608 - val_loss: 0.3850 - val_accuracy: 0.8280
Epoch 2/10
25000/25000 [==============================] - 50s 2ms/step - loss: 0.3228 - accuracy: 0.8638 - val_loss: 0.3802 - val_accuracy: 0.8318
Epoch 3/10
25000/25000 [==============================] - 50s 2ms/step - loss: 0.3160 - accuracy: 0.8678 - val_loss: 0.3819 - val_accuracy: 0.8319
Epoch 4/10
25000/25000 [==============================] - 52s 2ms/step - loss: 0.3105 - accuracy: 0.8708 - val_loss: 0.3719 - val_accuracy: 0.8366
Epoch 5/10
25000/25000 [==============================] - 67s 3ms/step - loss: 0.3045 - accuracy: 0.8728 - val_loss: 0.3775 - val_accuracy: 0.8322
Epoch 6/10
25000/25000 [==============================] - 69s 3ms/step - loss: 0.3004 - accuracy: 0.8745 - val_loss: 0.3754 - val_accuracy: 0.8346
Epoch 7/10
25000/25000 [==============================] - 63s 3ms/st